In [2]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


# Combine information schema, Attribute metadata and GCCT schema

In [11]:
import xlrd
output_name='2.5_mar26'
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory']


In [12]:
temp_df=pd.ExcelFile('GCCT Schema single excel2.5.xlsx').parse('Sheet1')
information_schema=pd.ExcelFile('/s/latluri1/DES/desqa_mar_26_info_schema.xlsx').parse('Sheet1')
attr_metadata=pd.ExcelFile('/s/latluri1/DES/desqa_mar_26_attr_meta.xlsx').parse('Sheet1')

In [13]:
temp_df[['Table','Logical Name']]=temp_df[['Table','Logical Name']].apply(lambda x: x.astype(str).str.lower())
attr_metadata[['EntityName','AttributeName']]=attr_metadata[['EntityName','AttributeName']].apply(lambda x: x.astype(str).str.lower())
information_schema[['TABLE_NAME','COLUMN_NAME']]=information_schema[['TABLE_NAME','COLUMN_NAME']].apply(lambda x: x.astype(str).str.lower())

In [14]:
df2=attr_metadata.merge(information_schema,how='outer',left_on=['EntityName','AttributeName'],right_on=['TABLE_NAME','COLUMN_NAME'],indicator=True)
df2=df2.rename(columns = {'_merge':'infor_schem_attr_metadata'})
df2.loc[:,'infor_schem_attr_metadata']=df2['infor_schem_attr_metadata'].replace('left_only', 'information_schema')
df2.loc[:,'infor_schem_attr_metadata']=df2['infor_schem_attr_metadata'].replace('right_only', 'attribute_metadata')
df2.loc[:,'infor_schem_attr_metadata']=df2['infor_schem_attr_metadata'].replace('both', 'combined_metadata')

def combined_column(r):
    keys_row=r.to_dict().keys()
    row=r.to_dict()
    a=row[keys_row[0]]
    b=row[keys_row[1]]
    if((pd.isnull(a)) & (np.logical_not(pd.isnull(b))) ):
        return(b)
    elif((pd.isnull(b)) & (np.logical_not(pd.isnull(a))) ):
        return(a)
    elif((np.logical_not(pd.isnull(b))) & (np.logical_not(pd.isnull(a)))):
        return(a)
    else:
        raise ValueError('Error in the code, please fix it')

def priority(r):
    if(r in tab_list):
        return("True")
    else:
        return("False")
        
    
#new.loc[:,'DATA_TYPE']=
df2["Table_infor_attr_combined"]=df2[['EntityName','TABLE_NAME']].apply(lambda r: combined_column(r), axis=1)
df2["col_infor_attr_combined"]=df2[['AttributeName','COLUMN_NAME']].apply(lambda r: combined_column(r), axis=1)
df3=temp_df.merge(df2,how='outer',left_on=['Table','Logical Name'],right_on=["Table_infor_attr_combined","col_infor_attr_combined"],suffixes=('_1','_2'),indicator=True )
df3["Table_combine"]=df3[['Table','Table_infor_attr_combined']].apply(lambda r: combined_column(r), axis=1)
df3["col_combine"]=df3[['Logical Name','col_infor_attr_combined']].apply(lambda r: combined_column(r), axis=1)
df3["priotity_combined"]=df3["Table_combine"].apply(lambda r: priority(r))
df3.loc[:,'_merge']=df3['_merge'].replace('left_only', 'GCCT_schema')
df3.loc[:,'_merge']=df3['_merge'].replace('right_only', 'combined_metadata')

In [20]:
writer = pd.ExcelWriter("GCCT_Schema_infor_schema_attr_meta_combined_"+output_name+".xlsx", engine = 'xlsxwriter')
df3.to_excel(writer, sheet_name="Sheet1", index=False)
writer.save()
writer.close()

In [21]:
stmnt='echo \"Combined file from GCCT schema, Information schema and attr metadata  '+output_name+'\" | mailx -a GCCT_Schema_infor_schema_attr_meta_combined_'+output_name+'.xlsx -s GCCT\ Schema\ GCCT_Infor_attr_'+output_name+' latluri1@ford.com'
print(stmnt)

echo "Combined file from GCCT schema, Information schema and attr metadata  2.5_mar26" | mailx -a GCCT_Schema_infor_schema_attr_meta_combined_2.5_mar26.xlsx -s GCCT\ Schema\ GCCT_Infor_attr_2.5_mar26 latluri1@ford.com


In [22]:
commands.getstatusoutput(stmnt)

(0, '')

In [16]:
df4=df3.copy()

for col in df4.columns:
    if(df4[col].dtypes=="object"):
             df4[col]=pd.factorize(df4[col],na_sentinel=-1)[0]

In [87]:
df4.corr()

,Additional data,Audit Enabled,Custom Attribute,Description,Display Name,Form location,Logical Name,Priority,Required Level,Schema Name,...,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME,infor_schem_attr_metadata,Table_infor_attr_combined,col_infor_attr_combined,_merge,Table_combine,col_combine,priotity_combined
Additional data,1.000000,-0.112019,0.157562,0.408146,0.419699,0.309902,0.347162,0.082945,0.126322,0.346550,...,NaN,NaN,NaN,-0.029117,0.020937,0.006083,-0.076122,0.150338,0.205297,0.050343
Audit Enabled,-0.112019,1.000000,0.247193,-0.122286,-0.143057,-0.156440,-0.057089,0.475700,0.284734,-0.057404,...,NaN,NaN,NaN,-0.395828,-0.302389,-0.598853,-0.246280,0.059656,-0.506347,0.273695
Custom Attribute,0.157562,0.247193,1.000000,0.156981,0.265437,0.287625,0.394675,0.256887,0.312728,0.397968,...,NaN,NaN,NaN,-0.112809,-0.062991,-0.302027,-0.500224,-0.093489,-0.132109,0.000034
Description,0.408146,-0.122286,0.156981,1.000000,0.629865,0.399038,0.510299,0.159151,0.176934,0.508726,...,NaN,NaN,NaN,-0.089423,0.013631,-0.013766,-0.072270,0.417822,0.298845,0.119357
Display Name,0.419699,-0.143057,0.265437,0.629865,1.000000,0.422418,0.679551,0.096148,0.156007,0.680272,...,NaN,NaN,NaN,-0.024795,0.047781,0.065526,-0.112018,0.257106,0.451646,0.050142
Form location,0.309902,-0.156440,0.287625,0.399038,0.422418,1.000000,0.319367,-0.039896,0.089152,0.320984,...,NaN,NaN,NaN,0.107449,0.184374,0.144655,-0.187065,0.128246,0.195520,-0.097047
Logical Name,0.347162,-0.057089,0.394675,0.510299,0.679551,0.319367,1.000000,0.127781,0.176471,0.993933,...,NaN,NaN,NaN,-0.103363,-0.007902,0.033381,-0.079376,0.346488,0.679251,0.066895
Priority,0.082945,0.475700,0.256887,0.159151,0.096148,-0.039896,0.127781,1.000000,0.426310,0.125711,...,NaN,NaN,NaN,-0.851907,-0.707786,-0.784608,0.128843,0.088535,-0.396339,0.913262
Required Level,0.126322,0.284734,0.312728,0.176934,0.156007,0.089152,0.176471,0.426310,1.000000,0.178130,...,NaN,NaN,NaN,-0.276162,-0.208257,-0.433061,-0.247606,0.033700,-0.237277,0.268199
Schema Name,0.346550,-0.057404,0.397968,0.508726,0.680272,0.320984,0.993933,0.125711,0.178130,1.000000,...,NaN,NaN,NaN,-0.099576,-0.004334,0.033765,-0.083534,0.345689,0.673612,0.063740
